In [0]:

from pyspark.sql.functions import col
user_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
file_location = f'/Workspace/Users/{user_name}/data/telco.csv'
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

from pyspark.sql.functions import trim, when 
#df_tc_dbl = df.withColumn('TotalCharges', trim(col("TotalCharges")))

df_tc_dbl = df.withColumn("TotalCharges", when(col('TotalCharges')==" " , None) \
          .otherwise(col('TotalCharges')))
df_tc_dbl = df_tc_dbl.withColumn('TotalCharges', col('TotalCharges').cast('double'))

display(df_tc_dbl)
     


In [0]:
%sh
#ls -al /
#lsb_release -a

ls -al /Workspace/Users/pawel.rubach@sgh.waw.pl/data

In [0]:
df_sel = df_tc_dbl.select('Contract', 'PhoneService', 'TotalCharges', 'MonthlyCharges', 'tenure')
#display(df_sel)
df_sel.show(5)
display(df_sel)

In [0]:
df_fil = df_sel.filter((df_sel['Contract'] == 'Month-to-month') & (df_sel['PhoneService'] == 'Yes')) 
# & - AND
# | - OR
# ~ - NOT
display(df_fil)

In [0]:
df_fil = df_sel.filter(df_sel['Contract'] == 'Month-to-month') 
df_fil.show()

In [0]:
from pyspark.sql.functions import asc, desc
df_fil = df_sel.filter((df_sel['Contract'] == 'Month-to-month') | (df_sel['TotalCharges'] < 60)) 
print(type(df_sel['Contract']))

#df_fil = df_fil.sort(df_sel['Contract'].desc(), df_fil['TotalCharges'].asc())
df_fil = df_fil.sort(desc(df_sel['Contract']), asc(df_fil['TotalCharges']))

df_fil.show()



In [0]:
df_grp = df_sel.groupBy('Contract', 'PhoneService').count()
#df_grp = df_sel.groupBy('Contract', 'PhoneService').avg('TotalCharges')
df_grp.show()

In [0]:
from pyspark.sql.functions import avg, count, round

#df_grp = df_sel.groupBy('Contract', 'PhoneService').agg(avg('TotalCharges'), count('*'))
df_grp = df_sel.groupBy('Contract', 'PhoneService').agg(round(avg('TotalCharges'), 2).alias('Avg TotalCharges'), count('*').alias('Count'))
#df_grp = df_sel.groupBy('Contract', 'PhoneService').avg('TotalCharges')
df_grp.show()

In [0]:
grp_results = df_grp.collect()
print(type(grp_results))
print(grp_results)
for row in grp_results:
    row_dict = row.asDict()
    print(type(row_dict))
    print(row_dict)
    print(row_dict['Avg TotalCharges'])
    avg_tc = row_dict.get('Avg TotalCharges')
    print(type(avg_tc))
    #print(row_dict.get('Avg TotalCharges'))
  